In [ ]:
from data import NYUDataset, RescaleDepth
import yaml
import sys
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import yaml
import os
import logging
from tqdm import tqdm
from torchvision.transforms import v2 as t
from torchvision import transforms
from torchvision.datasets import VisionDataset

from architecture import build_METER_model
from metrics import REL, RMSE, delta
import matplotlib.pyplot as plt

def load_config(config_path):
    """Load configuration from YAML file."""
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config('./config.yaml')

dataset = NYUDataset(
    root=config['data']['root'],
    test=True,
)
dataloader = DataLoader(dataset, 128)



/mnt/data/nyu/nyu_data/data


In [ ]:
import numpy as np
from torch import Tensor
from PIL import Image


def tensor_to_image(x: Tensor, rgb=False):
    # np_image = x.permute(0, 1).numpy()
    if x.get_device() >= 0:
        x = x.cpu()

    if rgb:
        x = x.permute((1, 2, 0))
    
    np_image = x.numpy()
    # 3. Convert to uint8 (for displaying as an image)
    np_image = (np_image).astype(np.uint8)

    # 4. Create PIL Image
    pil_image = Image.fromarray(np_image, "RGB" if rgb else None)

    # 5. Display using matplotlib inline in notebook
    if rgb:
        plt.imshow(pil_image)
    else:
        plt.imshow(pil_image, vmin=0, vmax=1000)
    plt.axis('off')
    plt.show()

In [3]:
def show_nth_dataset(model, dataset, n):
    img, depth = dataset[n]
    img = img.unsqueeze(0).to(device="cuda")
    depth = depth.unsqueeze(0).to(device="cuda")
    with torch.no_grad():
        output = model(img)
    print(img.shape)
    print(depth.shape)
    print(output.shape)
    tensor_to_image((img).squeeze(dim=(0,1)), True)
    tensor_to_image((depth).squeeze(dim=(0,1)))
    tensor_to_image((output).squeeze(dim=(0,1)))

In [1]:
from metrics import *

BATCH = 128

# Test case: Perfect prediction
pred = torch.ones(BATCH, 1, 192, 256) * 5.0  # 5m depth everywhere
target = torch.ones(BATCH, 1, 192, 256) * 5.0

print(RMSE(pred, target))  # Should be 0.0
print(REL(pred, target))   # Should be 0.0
print(delta(pred, target)) # Should be 1.0

# Test case: Off by 1m
pred = torch.ones(BATCH, 1, 192, 256) * 6.0
print(REL(pred, target))   # Should be 0.2 (|6-5|/5)

tensor(0.)
tensor(0.)
tensor(1.)
tensor(0.2000)


In [6]:
from evaluate import load_checkpoint


model = build_METER_model("cuda", "s")
model = model.to("cuda")
x = load_checkpoint(model, './models/build_model_best_nyu_s')

In [7]:
max_out = -1
max_data = -1
min_out = 10000
min_data = 10000
for r, d in dataloader:
    max_data = max(max_data, torch.max(d).float().item())
    min_data = min(min_data, torch.min(d).float().item())
    with torch.no_grad():
        output = model(dataset[0][0].unsqueeze(0).to(device="cuda"))
    max_out = max(max_out, torch.max(output).float().item())
    min_out = min(min_out, torch.min(output).float().item())
print("max_out", max_out)
print("max_data", max_data)
print("min_out", min_out)
print("min_data", min_data)

max_out 457.4888000488281
max_data 995.2000122070312
min_out 146.2028350830078
min_data 71.30000305175781


In [8]:
from loss import balanced_loss_function


def evaluate(model: nn.Module, dataloader, criterion, device):
    """Validate for one epoch."""
    model.eval()
    total_samples = 0

    metrics = []

    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Validation")
        
        for images, targets in progress_bar:
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, targets)

            metrics.append({
                "RMSE": RMSE(outputs, targets).item(),
                "REL": REL(outputs, targets).item(),
                "delta": delta(outputs, targets).item(),
                "loss_depth": loss[0].item(),
                "loss_grad": loss[1].item(),
                "loss_normal": loss[2].item(),
                "loss_ssim": loss[3].item(),
            })
            
            batch_size = images.size(0)
            total_samples += batch_size

    return metrics

metrics = evaluate(model, dataloader, balanced_loss_function(device="cuda", dtype=torch.float32), "cuda")
df = pd.DataFrame(metrics).mean()
df.to_csv("./metrics.csv")
print(df)


Validation: 100%|██████████| 6/6 [00:10<00:00,  1.80s/it]

RMSE            88.229588
REL              0.240335
delta            0.477409
loss_depth      71.435059
loss_grad      210.197385
loss_normal     36.093906
loss_ssim       19.531841
dtype: float64
